In [2]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from IPython.display import display, Latex
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import Sampler
from qiskit import transpile
import numpy as np
from qiskit_ibm_runtime import SamplerV2 as Sampler

from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService (channel="ibm_quantum",
                                token="304636e08832e99bf5cd66f58d0f1180f193cd79e97987083aafe0ba33cef2f9ceb8ac2bba91aa24b612787451d32e9dc7206edf2e4347ba88443ba5261f04a6")

QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token="304636e08832e99bf5cd66f58d0f1180f193cd79e97987083aafe0ba33cef2f9ceb8ac2bba91aa24b612787451d32e9dc7206edf2e4347ba88443ba5261f04a6",
                                  overwrite=True)
import pennylane as qml
from qiskit_aer import AerSimulator
from qiskit_aer import Aer
qubits = 4
dev = qml.device("qiskit.aer", wires=qubits, backend="aer_simulator")


print(Aer.backends())

from pennylane import numpy as pnp
from qiskit_ibm_runtime import QiskitRuntimeService

import pennylane as qml

# Obtaining the Hamiltonian for H2 from PennyLane QChem dataset
[dataset] = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")
H = dataset.hamiltonian
qubits = 4

# Initialize QiskitRuntimeService
service = QiskitRuntimeService()

backends = service.backends()
for backend in backends:
    print(backend.name)

backend = Aer.get_backend("aer_simulator")

try:
    # Our device supports a maximum of 31 qubits
    NUM_QUBITS_SUPPORTED = 31
    dev = qml.device("qiskit.remote", wires=NUM_QUBITS_SUPPORTED, backend=backend)
except Exception as e:
    print(e)

def four_qubit_ansatz(theta):
    # initial state 1100:
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)

    # change of basis
    qml.RX(pnp.pi / 2, wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)

    qml.CNOT(wires=[3, 2])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[1, 0])

    qml.RZ(theta, wires=0)

    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[3, 2])

    # invert change of basis
    qml.RX(-pnp.pi / 2, wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)

from pennylane_qiskit import qiskit_session


@qml.qnode(dev)
def cost_fn(theta):
    four_qubit_ansatz(theta)
    return qml.expval(H)


max_iterations = 10
theta = pnp.array(0.0, requires_grad=True)
opt = qml.GradientDescentOptimizer(stepsize=0.4)
energies = []


for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)
    energies.append(prev_energy)
    print(f"第 {n+1} 步: E = {prev_energy:.8f} Ha")


dev1 = qml.device("default.qubit", wires=4)
shots = 8000
dev2 = qml.device("qiskit.aer", wires=4, shots=shots)


@qml.qnode(dev1)
def cost_fn_1(theta):
    four_qubit_ansatz(theta)
    return qml.expval(H)


@qml.qnode(dev2)
def cost_fn_2(theta):
    four_qubit_ansatz(theta)
    return qml.expval(H)


# we can also use the qnode to draw the circuit
import matplotlib.pyplot as plt

qml.draw_mpl(cost_fn_1, decimals=2)(theta=1.0)
plt.show()

stepsize = 0.4
max_iterations = 10
opt = qml.GradientDescentOptimizer(stepsize=stepsize)
theta_1 = pnp.array(0.0, requires_grad=True)
theta_2 = pnp.array(0.0, requires_grad=True)
energies_1 = []
energies_2 = []
for n in range(max_iterations):
    theta_1, prev_energy_1 = opt.step_and_cost(cost_fn_1, theta_1)
    theta_2, prev_energy_2 = opt.step_and_cost(cost_fn_2, theta_2)
    print(prev_energy_1, prev_energy_2)
    energies_1.append(prev_energy_1)
    energies_2.append(prev_energy_2)

plt.plot(energies_1, color="r", label="default.qubit")
plt.plot(energies_2, color="b", label="qiskit.aer")

# min energy = min eigenvalue
min_energy = min(qml.eigvals(H))
z = [min_energy] * max_iterations

plt.plot(z, "--", color="k", label="Exact answer")
plt.xlabel("VQE iterations")
plt.ylabel("Energy (Ha)")
plt.legend()
plt.show()


[AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator')]


BlockingIOError: [Errno 11] Unable to synchronously open file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')